<a href="https://colab.research.google.com/github/ashikita/qir-toolbox/blob/main/xml-lang-add/tsv-lang-add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
TSV(@xml:lang 補完) 一括処理スクリプト
- カレントディレクトリの .txt ファイルすべてを対象
- @xml:lang 列で空のセルのみ、左隣の値から ja/en を推定して補完
- 出力: <元名>_add_xml-lang.txt
- 実行結果として総補完件数と処理時間を標準出力へ表示
"""

import os
import re
import glob
import time
import csv
import pandas as pd
import numpy as np

def main():
    start = time.perf_counter()

    # 作業対象の .txt ファイル一覧（昇順）
    txt_files = sorted(glob.glob("*.txt"))
    if not txt_files:
        print("[INFO] 対象となる .txt ファイルが見つかりません。")
        return

    # 日本語文字（ひらがな・カタカナ（全角/半角）・CJK統合漢字）
    jp_char_pattern = re.compile(r"[\u3040-\u309F\u30A0-\u30FF\uFF65-\uFF9F\u4E00-\u9FFF]")
    # 「英字のみ」とみなす許容記号（空白、ハイフン、ピリオド、カンマ、クォート、アンパサンド、スラッシュ、括弧）
    en_only_pattern = re.compile(r"^[A-Za-z\s\-\.,'&/()]+$")

    total_updates = 0
    total_files = 0

    for in_path in txt_files:
        # 入力: TSV で、空文字を空値として保持（NAにしない）
        # 文字コードはUTF-8(BOMあり/なし)を想定
        try:
            df = pd.read_csv(
                in_path,
                sep="\t",
                engine="python",
                dtype=str,
                quotechar='"',
                quoting=csv.QUOTE_MINIMAL,
                keep_default_na=False,  # 空文字を NaN にしない
                na_filter=False         # 既定の NaN 判定を無効化
            )
        except Exception as e:
            print(f"[WARN] 読み込み失敗: {in_path} -> {e}")
            continue

        # ヘッダー名が @xml:lang で終わる列を特定
        cols = list(df.columns)
        xml_cols = [c for c in cols if isinstance(c, str) and c.endswith("@xml:lang")]

        if not xml_cols:
            # @xml:lang 列がないファイルはスキップ
            print(f"[INFO] スキップ（@xml:lang 列なし）: {in_path}")
            continue

        file_updates = 0

        for col in xml_cols:
            col_idx = df.columns.get_loc(col)
            if col_idx == 0:
                # 仕様上、@xml:lang の左隣が存在しないことはない想定だが、安全対策
                continue

            # 左隣の値（対象文字列）と、@xml:lang の現在値
            left_series = df.iloc[:, col_idx - 1].astype(str)
            lang_series = df.iloc[:, col_idx].astype(str)

            # 補完対象マスク: @xml:lang が空 かつ 左隣が空でない
            m_missing_lang = lang_series.eq("")
            m_left_has_val = left_series.ne("")
            m_target = m_missing_lang & m_left_has_val

            if not m_target.any():
                continue  # この列での補完対象なし

            # 日本語文字を含むか（ベクトル化）
            m_has_jp = left_series.str.contains(jp_char_pattern, na=False)
            # 英字のみか（ベクトル化）
            m_is_en_only = left_series.str.fullmatch(en_only_pattern, na=False)

            # ルールに基づく補完マスク
            m_fill_ja = m_target & m_has_jp
            m_fill_en = m_target & (~m_has_jp) & m_is_en_only
            # それ以外は不確実 -> 変更しない（空のまま）

            # 補完の適用
            updates_ja = int(m_fill_ja.sum())
            updates_en = int(m_fill_en.sum())
            if updates_ja > 0:
                df.loc[m_fill_ja, col] = "ja"
            if updates_en > 0:
                df.loc[m_fill_en, col] = "en"

            file_updates += (updates_ja + updates_en)

        # 出力ファイル名: 末尾に _add_xml-lang を付与
        root, ext = os.path.splitext(in_path)
        out_path = f"{root}_add_xml-lang{ext or '.txt'}"

        try:
            df.to_csv(
                out_path,
                sep="\t",
                index=False,
                encoding="utf-8",
                quotechar='"',
                quoting=csv.QUOTE_MINIMAL,
                lineterminator="\n",
            )
        except Exception as e:
            print(f"[WARN] 書き込み失敗: {out_path} -> {e}")
            continue

        print(f"[DONE] {in_path} -> {out_path} | 補完 {file_updates} 件")
        total_updates += file_updates
        total_files += 1

    elapsed = time.perf_counter() - start
    print("==============================================")
    print(f"[RESULT] 処理ファイル数: {total_files}")
    print(f"[RESULT] 総補完件数  : {total_updates}")
    print(f"[RESULT] 総処理時間  : {elapsed:.3f} 秒")

if __name__ == "__main__":
    main()

[DONE] mmd-1967725517024443569.txt -> mmd-1967725517024443569_add_xml-lang.txt | 補完 1677 件
[RESULT] 処理ファイル数: 1
[RESULT] 総補完件数  : 1677
[RESULT] 総処理時間  : 0.217 秒
